#Running simple example through EC2

#### start downloading linking_EC2

Tutorial for running liknking_EC2 see:
https://github.com/JonasWallin/linkingEC2/blob/master/script/running%20MPI4py.ipynb

In [1]:
%%bash
. ~/.bashrc
pip install  --upgrade git+https://git@github.com/JonasWallin/linkingEC2

  Cloning https://git@github.com/JonasWallin/linkingEC2 to /var/folders/ct/zpjk2t696k5bv03sqq9xrpd00000gn/T/pip-FTtSJW-build
Requirement already up-to-date: boto in /Users/jonaswallin/anaconda/lib/python2.7/site-packages (from linkingEC2==0.1)
  Found existing installation: linkingEC2 0.1
    Uninstalling linkingEC2-0.1:
      Successfully uninstalled linkingEC2-0.1
  Running setup.py install for linkingEC2


In [23]:
from linkingEC2 import LinkingHandler
from ConfigParser import ConfigParser

config = ConfigParser()
starfigconfig_folder = "/Users/jonaswallin/.starcluster/"
config.read(starfigconfig_folder + "config")

acess_key_id     = config.get('aws info', 'aws_access_key_id'    , 0)
aws_secret_key   = config.get('aws info', 'aws_secret_access_key', 0)
aws_region_name  = config.get('aws info', 'aws_region_name'      , 0)
my_key_loc       = config.get('key mykeyABC', 'key_location',0)


linker = LinkingHandler(aws_secret_access_key = aws_secret_key,
                        aws_access_key_id     = acess_key_id,
                        aws_region_name       = aws_region_name,
                        key_location          = my_key_loc,
                        key_name              = 'mykeyABC' )

##### start the actual cluster

In [24]:

linker.start_cluster('ami-d05e75b8', 't2.micro', ['linking_EC2'],1)
print( linker.get_ssh_login() )



waiting for instances 0  to be running: 
******

checking if ssh into node001 works:
**
collecting node001 number of processes:done
copying ~/.ssh/mykeyABC.rsa to external ~/.ssh/id_rsa
copying files to node001 done
disable StrictHostKeyChecking  in node001:appending to ~/nodefile in node001 done
appending to /etc/hosts in node001 done
ssh  -i ~/.ssh/mykeyABC.rsa -o 'StrictHostKeyChecking no'  ubuntu@ec2-52-0-110-236.compute-1.amazonaws.com


In [25]:
PACKAGES_APT = [' libatlas3-base',
                'libatlas-base-dev',
                'python-dev', 
                'openmpi-bin', 
                'libopenmpi-dev', 
                'python-numpy',
                'python-sklearn',
               'python-matplotlib',
               'git',
                'python-scipy',
               'r-base',
               'r-base-core']
PACKAGES_PIP = ['cython',
                'mpi4py',
                'simplejson',
               'rpy2']

#Adding later version of R
#http://philipp-burckhardt.com/2014/05/25/installing-r-rstudio-on-ubuntu/
command = 'sudo add-apt-repository "deb http://cran.rstudio.com/bin/linux/ubuntu trusty/"'
linker.send_command_ssh(command = 'gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9')
linker.send_command_ssh(command = 'gpg -a --export 51716619E084DAB9 | sudo apt-key add -')
linker.send_command_ssh(command = command)



linker.apt_install(PACKAGES_APT)

#problem with memory installing scipy:
#http://naokiwatanabe.blogspot.se/2014/12/install-numpy-schipy-matplotlib-and-etc.html
linker.send_command_ssh(command = 'sudo /bin/dd if=/dev/zero of=/var/swap.1 bs=1M count=1024')
linker.send_command_ssh(command = 'sudo /sbin/mkswap /var/swap.1')
linker.send_command_ssh(command = 'sudo /sbin/swapon /var/swap.1')
linker.pip_install('-U scipy')

linker.send_command_ssh(command = 'sudo swapoff /var/swap.1')
linker.send_command_ssh(command = 'sudo sudo rm /var/swap.1')



linker.pip_install(PACKAGES_PIP)
linker.pip_install(['git+https://git@github.com/JonasWallin/BayesFlow'])
import os
os.system('say "your packages is downloaded"')

sshing in: gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9 :
sshing in: gpg -a --export 51716619E084DAB9 | sudo apt-key add - :
sshing in: sudo add-apt-repository "deb http://cran.rstudio.com/bin/linux/ubuntu trusty/" :
update apt-get for node001 done
upgrade apt-get for node001 done
installing the packages :  libatlas3-base libatlas-base-dev python-dev openmpi-bin libopenmpi-dev python-numpy python-sklearn python-matplotlib git python-scipy r-base r-base-core
installing package for node001 done
sshing in: sudo /bin/dd if=/dev/zero of=/var/swap.1 bs=1M count=1024 :
sshing in: sudo /sbin/mkswap /var/swap.1 :
sshing in: sudo /sbin/swapon /var/swap.1 :
installing the packages : python-pip
installing package for node001 done
installing the packages : -U scipy
installing package for node001 done
sshing in: sudo swapoff /var/swap.1 :
sshing in: sudo sudo rm /var/swap.1 :
installing the packages : cython mpi4py simplejsonrpy2
installing package for node001 failed:

ssh     -i ~/.s

0

In [29]:
linker.send_command_ssh( command = 'wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/article_simulatedata.py')
linker.send_command_ssh( command = 'wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/article_estimate_largerdata1_mpi.py')

sshing in: wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/article_simulatedata.py :
sshing in: wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/article_estimate_largerdata1_mpi.py :


0

# Terimante the cluster:

In [19]:
linker.terminate_cluster()